In [1]:
!pip install  openai==0.28
!pip install tenacity


**Medical Question-Ansswering bot**

Following notebook presents Fine Tuning a task specific model by Generating Example data for the particular task.

Task Description :

Developing a task-specific chatbot for leveraging the medical domain knowledge to understand and response different medical queries. Because chat-gpt models are trained for geneal data. Fine tuning with a specific domain data will leverage the domain specific-task. Our will be trained to response following keypoints as the medical assistant.


*   Syptoms
*   diagnoses
*   treatments
*   medications
*   general health advice

Steps :


1.   Generate example datasets with providing well explained prompt and correctly defined system content

2.   Fine-tuning gpt-3.5 model to engaging users to retrieve medical informations



In [2]:
import os
import openai
import random
from tenacity import retry, stop_after_attempt, wait_exponential
from pprint import pprint

openai.api_key = ""

N_RETRIES = 3
TEMPERATURE = .5


@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def generate_example(prompt:str , prev_examples:list, temperature : float) -> str:

  """
  Generate single example of prompt/response pair to fine tune the mdel

  Parameters :
  - prompt (str) : Proper prompt to generate questions/responses
  - prev_examples (list) : list of created examples (training data)
  - temperature (float) : accept any valur from 0 to 1. define the random of the generated reponses by openai models


  Returns :
  - str: Generated example in the format of prompt / response
  """

  # defining sytem role
  messages=[
      {
          "role": "system",
          "content": f"You are generating data to train a machine learning model for a Medical Question-Answering System.\n\nYour task is to design a system that can intelligently handle user queries related to medical topics. Each example should be in the prompt/response pair format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nGenerate one prompt/response pair per turn, gradually increasing the complexity while ensuring diversity.\n\nCreate unique, high-quality samples that cover a wide range of medical scenarios. The ultimate goal is to train a system that excels in providing accurate and informative responses to medical queries.\n\nHere is the type of Medical Question-Answering System we want to develop:\n`{prompt}`"
      }
  ]

  if len(prev_examples) > 5:
      # limiting nb of examples to measure the complexity to generate next prompt
      if len(prev_examples) > 3:
          prev_examples = random.sample(prev_examples, 5)

      # append examples to the previous examples
      for example in prev_examples:
          messages.append({
              "role": "assistant",
              "content": example
          })

  #OpenAI Chat Completion API Call
  response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=messages,
      temperature=TEMPERATURE,
      max_tokens=1000,
  )

  return response.choices[0].message['content']




prompt = "A model which is a Medical Question-Answering System capable of understanding and responding to various medical queries, covering symptoms, diagnoses, treatments, medications, and general health advice."
number_of_examples = 100

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt , prev_examples, TEMPERATURE)
    prev_examples.append(example)

pprint(prev_examples)

Generating example 0
Generating example 1
Generating example 2
Generating example 3
Generating example 4
['prompt\n'
 '-----------\n'
 'What are the symptoms of the common cold?\n'
 '-----------\n'
 '\n'
 'response\n'
 '-----------\n'
 'Common symptoms of a cold include a runny or stuffy nose, sore throat, '
 'cough, congestion, slight body aches, mild headache, sneezing, low-grade '
 'fever, and mild fatigue. The symptoms can vary from person to person and not '
 'everyone will experience all of these symptoms.\n'
 '-----------',
 'prompt\n'
 '-----------\n'
 'What is the difference between type 1 and type 2 diabetes?\n'
 '-----------\n'
 '\n'
 'response\n'
 '-----------\n'
 'Type 1 and Type 2 diabetes are both chronic diseases that affect the way '
 'your body regulates blood sugar, or glucose. \n'
 '\n'
 'Type 1 diabetes is an autoimmune disease. The immune system attacks and '
 "destroys cells in the pancreas, where insulin is made. It's unclear what "
 'causes this attack. About 1

In [3]:
def generate_system_message(prompt: str) -> str:
  """
  Generate system message for particular model for given user prompt

  Parameters :
  - prompt (str): user prompt to generate system message

  Returns :
  - str : generated system message

  """

  #API call to gpt-4 to generate system message
  response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
        {
          "role": "system",
          "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
        },
        {
            "role": "user",
            "content": prompt.strip(),
        }
      ],
      temperature=TEMPERATURE, # control randomness
      max_tokens=500, # limit the length of generated system message
  )

  return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Given your medical query, the system will provide a detailed response, covering potential symptoms, diagnoses, treatments, medications, and general health advice.`. Feel free to re-run this cell if you want a better result.


**Following, process the generated examples and convert them into desired formats that allows to fine tune gpt-3.5**

In [ ]:
import json
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('----')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples.')

# Initialize list to store training examples
training_examples = []

# Create training examples in the format required for GPT-3.5 fine-tuning
for index, row in df.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Save training examples to a .jsonl file
with open('training_smples.jsonl', 'w') as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')

Let's uplod the created examples to finetune the chat-GPT (gpt-3.5-turbo) model. The fine-tuned model will be able to interact for user queries speciifed in medical domain to retrieve accurate responses.

In [5]:
file_id = openai.File.create(
  file=open("/content/training_examples.jsonl", "rb"),
  purpose='fine-tune'
).id

In [6]:
# Intializing the fine tuning job to rtain the model
job = openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")

job_id = job.id

In [7]:
openai.FineTuningJob.list_events(id=job_id, limit=10)

<OpenAIObject list at 0x7ff472b59120> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-A9rLXakgw2oy3zrqsKHdRxml",
      "created_at": 1705511581,
      "level": "info",
      "message": "Validating training file: file-JYMc0AThMExZGzbKaDeG2Q3a",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-fCueX2t1NBiPIepH7WyWptM9",
      "created_at": 1705511581,
      "level": "info",
      "message": "Created fine-tuning job: ftjob-tMoTcPlv8bznmsuRH9esQVIm",
      "data": {},
      "type": "message"
    }
  ],
  "has_more": false
}



---


According to the above results, we can monitor events related to fine-tuning


job


1.   Validating training file
2.   Created Fine tuning job

System is ensuring the validity of the training data also it confirmed the intiating of fine-tune job by giving them ids and created_at parametrs


FOllowing code showcase Inference example.

Using the following code the fine-tuned model can be used as a medical assistant to retrieve informations. It has ability to provide information related to symptoms, diagnose, treatments.

In [ ]:
# retrieve the trained model to chat with
model_name_pre_object = openai.FineTuningJob.retrieve(job_id)
model_name = model_name_pre_object.fine_tuned_model
print(model_name)

In [ ]:
response = openai.ChatCompletion.create(
    model=model_name,
    messages=[
      {
        "role": "system",
        "content": system_message,
      },
      {
          "role": "user",
          "content": df['prompt'].sample().values[0],
      }
    ],
)

response.choices[0].message['content']